In [1]:
# the packages
import pandas as pd
import numpy as np
import networkx as nx

# import my own helper functions
from read import read_sims_result
from clean import cleanup_0IR_exp
from clean import cleanup_network

# pearson correlation coeffcient
from scipy.stats.stats import pearsonr

# deque
from collections import deque

# deep copy
from copy import deepcopy

# random [0,1)
from random import random

# Page Rank
from networkx.algorithms.link_analysis.pagerank_alg import pagerank, pagerank_numpy, pagerank_scipy

# distance
from networkx.algorithms.shortest_paths.generic import shortest_path_length

# logistic regression
from sklearn.linear_model import LogisticRegression

# cross validation
from sklearn.model_selection import KFold

# confusion matrix
from sklearn.metrics import confusion_matrix

# plot
import matplotlib.pyplot as plt

In [2]:
# independent variables
independent = ["deposits", "cash", "assets", "credit available", "wealth", "leverage", 
         "dummy-0-leverage",
         "wealth-lag", "deposits-lag", "cash-lag", "assets-lag", "leverage-lag", 
         "credit-available-lag", "credit-issued-lag", "dummy-0-leverage-lag",
         "over-leverage-frequency"]

In [3]:
# ###########################
# Read OIR results, and fit the model
# ###########################
df0 = read_sims_result("/Users/xcheng/Documents/Oberlin/Summer2/DataAnalysis/data/0622/0IR300s", 32)
df0c = cleanup_0IR_exp(df0, numNode=32, numPeriod=15, numSim=100, balanced=True)

X = df0c[independent]
y = df0c["default-next"]

final = LogisticRegression(penalty="l1", C=0.007)
final.fit(X,y)

LogisticRegression(C=0.007, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [4]:
# make sure no defaults in 0IR
# sum(df0[df0["defaults due to interest"]
#     +df0["defaults due to negative wealth"]
#     +df0["defaults due to deposit shock"] == 0].loc[:,"dot0":"dot30"].values)

In [60]:
# ###########################
# Read & process positive IR results
# ###########################
df_1 = read_sims_result("/Users/xcheng/Documents/Oberlin/Summer2/DataAnalysis/data/0625/1IR", 32)
mx_1n = cleanup_network(df_1, numNode=32, numPeriod=15, numSim=50)
df_1c = cleanup_0IR_exp(df_1, numNode=32, numPeriod=15, numSim=50)

In [61]:
def create_edge_weight(N, dff, mid):
    """
    Calculate weight for edges
    Each debt is divided by lenders' wealth (w/o haircut)，
    The result number r is scaled to [0, 1) using g(r)=r/(mid+r)
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    df: Pandas dataframe (no cleanup)
        where we get banks' wealth (w/o haircut)
    mid: int
        the debt-to-wealth ratio resulting in 50% probability of spreading default
    
    Returns
    ----------
    WN: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        new weighted debt adjacency matrices
    """
    
    WN = np.copy(N)
    simNum, periodNum, bankNum, _= N.shape
    dff["book_wealth"] = (dff["assets"] + dff["cash"] + dff["debt owed"] 
                         - dff["debt to pay"] - dff["deposits"])
    
    for s in range(simNum):
        for p in range(1,periodNum+1):
            for lender in range(bankNum):
                w = dff[np.array(dff["sim#"]==s) &
                        np.array(dff["period"]==p) & 
                        np.array(dff["bankID"]==lender)
                       ]["book_wealth"].values[0]
                
                # helper function
                def f(a):
                    if a > 0: # there is debt
                        if w > 0: # positive wealth
                            t = a/w
                            return t/(t+mid)
                        else: # 0 or negative wealth
                            return 100/(100+mid)
                    else: # no debt or weird data 
                        return 0
                
                WN[s, p-1, :, lender] = [f(k) for k in WN[s, p-1, :, lender]]
                    
    return WN

In [62]:
def create_node_weight(N, dff, model, variables):
    """
    Calculate weight for nodes
    Each debt is multiplied by lenders' predicted default probability
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    dff: Pandas dataframe (yes cleanup)
        where we get bank's balance sheet info
    model: model for default probability
        scikit learn LogisticRegression
    variables: a list of strings
        independent variables for the model
    
    Returns
    ----------
    WN: 3D numpy array [n_simulations, n_periods, n_banks]
        array of predicted probability of default
    """
    
    simNum, periodNum, bankNum, _= N.shape
    WN = np.empty((simNum, periodNum, bankNum))
    WN.fill(-1)
    
    for s in range(simNum):
        for p in range(2,periodNum):
            for b in range(bankNum):
                X = dff[np.array(dff["sim#"]==s) &
                        np.array(dff["period"]==p) & 
                        np.array(dff["bankID"]==b)
                       ][variables].values
                if X.any():
                    predicted_default_probability = model.predict_proba(X)[0][1]
                    WN[s, p-1, b] = predicted_default_probability
                    
    return WN

In [63]:
edge_weights_all = create_edge_weight(mx_1n, df_1, 0.6)
node_weights_all = create_node_weight(mx_1n, df_1c, final, independent)

Empirically, I discover that it does not matter 0.6 is. I experimented with serveral numbers between 0 and 1.

In [64]:
# visualize the debt adjacency matrix in a 2d graph
# plt.figure(figsize=(10,10))
# plt.imshow(edge_weights_all[0,1], interpolation='nearest')
# plt.show()

In [65]:
def random_walk_single_1a(N, solvent, iterations=10):
    """
    NEW NEW modified random walk algorithm
    at the begining of each round, flip coin to cause one solvent bank to default
    
    This one is aggressive
    [[18054    25] ---0625/1IR
     [  158     3]]
    [[17926    30] ---0625/2IR
     [  160     3]]
    
    Parameters
    ----------
    N: 2D numpy array [n_borrowers, n_lenders]
        debt adjacency matrices 
    solvent: list 
        solvent[solvent bank] = predicted default probability
        solvent[insolvent bank] = -1
    iterations: int
        number of iterations
    
    Returns
    ----------
    result: dict {BankID : # of default}
        index for the bank's default probability
    """
    num_bank, _ = N.shape
    G = nx.DiGraph(N)
    nextDefault = deque()
    defaulted = frozenset([i for i in range(num_bank) if solvent[i] == -1])
    tempDefault = set(defaulted)
    probDefault = {i:solvent[i] for i in range(num_bank) if solvent[i] >= 0}
    result = dict(zip(list(probDefault.keys()), [0 for _ in probDefault.keys()]))
    def coin(p=0.5):
        return random() < p
    
    for _ in range(iterations):
        for b in probDefault.keys():

            if coin(probDefault[b]): 
                nextDefault.append(b)
            while len(nextDefault) > 0: # queue not empty
                n = nextDefault.popleft() # next on the queue
                if n not in tempDefault: # not aleady defaulted
                    tempDefault.add(n)
                    result[n] += 1
                    for s in G.successors(n): # creditors of n
                        if s not in tempDefault and coin(G.edges[(n,s)]['weight']):
                            nextDefault.append(s)
            tempDefault = set(defaulted)
            
    return result

In [66]:
def random_walk_single_1b(N, solvent, iterations=10):
    """
    NEW NEW modified random walk algorithm
    at the begining of each round, flip coin to cause one solvent bank to default
    during each round, we can add 1 multiple times
    
    This one is also aggressive
    [[18052    27] ---0625/1IR
     [  157     4]]
    [[17927    29] ---0625/2IR
     [  160     3]]
    
    Parameters
    ----------
    N: 2D numpy array [n_borrowers, n_lenders]
        debt adjacency matrices 
    solvent: list 
        solvent[solvent bank] = predicted default probability
        solvent[insolvent bank] = -1
    iterations: int
        number of iterations
    
    Returns
    ----------
    result: dict {BankID : # of default}
        index for the bank's default probability
    """
    num_bank, _ = N.shape
    G = nx.DiGraph(N)
    nextDefault = deque()
    probDefault = {i:solvent[i] for i in range(num_bank) if solvent[i] >= 0}
    result = dict(zip(list(probDefault.keys()), [0 for _ in probDefault.keys()]))
    def coin(p=0.5):
        return random() < p
    
    for _ in range(iterations):
        for b in probDefault.keys():
            if coin(probDefault[b]): 
                nextDefault.append(b)
            while len(nextDefault) > 0: # queue not empty
                n = nextDefault.popleft() # next on the queue
                result[n] += 1 # default count ++
                for s in G.successors(n): # creditors of n
                    if coin(G.edges[(n,s)]['weight']):
                        nextDefault.append(s)
                        
    return result

In [67]:
def random_walk_single_2a(N, solvent, iterations=10):
    """
    NEW NEW NEW modified random walk algorithm
    at the begining of each round, flip multiple coins for each solvent bank to make them default
    
    This one is useless (very progressive):
    [[18079     0] ---0625/1IR
     [  161     0]]
    [[17956     0] ---0625/2IR
     [  163     0]]
    
    Parameters
    ----------
    N: 2D numpy array [n_borrowers, n_lenders]
        debt adjacency matrices 
    solvent: list 
        solvent[solvent bank] = predicted default probability
        solvent[insolvent bank] = -1
    iterations: int
        number of iterations
    
    Returns
    ----------
    result: dict {BankID : # of default}
        index for the bank's default probability
    """
    num_bank, _ = N.shape
    G = nx.DiGraph(N)
    nextDefault = deque()
    defaulted = frozenset([i for i in range(num_bank) if solvent[i] == -1])
    tempDefault = set(defaulted)
    probDefault = {i:solvent[i] for i in range(num_bank) if solvent[i] >= 0}
    result = dict(zip(list(probDefault.keys()), [0 for _ in probDefault.keys()]))
    def coin(p=0.5):
        return random() < p
    
    for _ in range(iterations):
        for b in probDefault.keys():
            if coin(probDefault[b]): 
                nextDefault.append(b)

        while len(nextDefault) > 0: # queue not empty
            n = nextDefault.popleft() # next on the queue
            if n not in tempDefault: # not aleady defaulted
                tempDefault.add(n)
                result[n] += 1
                for s in G.successors(n): # creditors of n
                    if s not in tempDefault and coin(G.edges[(n,s)]['weight']):
                        nextDefault.append(s)
        tempDefault = set(defaulted)
    
    return result

In [68]:
def random_walk_single_2b(N, solvent, iterations=10):
    """
    NEW NEW NEW modified random walk algorithm
    at the begining of each round, flip multiple coins for each solvent bank to make them default
    during each round, we can add 1 multiple times
    
    This one is progressive:
    [[18048    31] ---0625/1IR
     [  157     4]]
    [[17926    30] ---0625/2IR
     [  157     6]]
    
    Parameters
    ----------
    N: 2D numpy array [n_borrowers, n_lenders]
        debt adjacency matrices 
    solvent: list 
        solvent[solvent bank] = predicted default probability
        solvent[insolvent bank] = -1
    iterations: int
        number of iterations
    
    Returns
    ----------
    result: dict {BankID : # of default}
        index for the bank's default probability
    """
    num_bank, _ = N.shape
    G = nx.DiGraph(N)
    nextDefault = deque()
    probDefault = {i:solvent[i] for i in range(num_bank) if solvent[i] >= 0}
    result = dict(zip(list(probDefault.keys()), [0 for _ in probDefault.keys()]))
    def coin(p=0.5):
        return random() < p
    
    for _ in range(iterations):
        for b in probDefault.keys():
            if coin(probDefault[b]): 
                nextDefault.append(b)
        while len(nextDefault) > 0: # queue not empty
            n = nextDefault.popleft() # next on the queue
            result[n] += 1
            for s in G.successors(n): # creditors of n
                if coin(G.edges[(n,s)]['weight']):
                    nextDefault.append(s)
                    
    return result

In [69]:
def customized_random_walk_exp(N, solvent, random_walk, iterations=10):
    """
    NEW NEW NEW modified random walk algorithm
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    solvent: 3D numpy array [n_simulations, n_periods, n_borrowers, n_lenders] 
        predicted default probability OR -1
    iterations: int
        number of iterations
    
    Returns
    ----------
    result: 1D numpy array
        index for the bank's default probability   
    """
    n_s, n_p, n_b, _ = N.shape
    s_s, s_p, s_b = solvent.shape
    big_result = []
    
    if n_s != s_s or n_p != s_p or n_b != s_b:
        raise ValueError('Two arrays have incompatible sizes.')
        
    for i, j in np.ndindex((n_s,n_p)):
        big_result.extend(random_walk(N[i,j],
                                      solvent[i,j],
                                      iterations=iterations
                                     ).values())
        
    return np.array(big_result)

In [70]:
overall_prediction = customized_random_walk_exp(edge_weights_all, node_weights_all, random_walk_single_2b, 100)

In [71]:
len(overall_prediction)

18240

In [72]:
# ###########################
# pearson correlation coeffcient is not very big 
# it gets slightly bigger with lots of iterations
# ###########################
pearsonr(overall_prediction, df_1c["default-next"])

(0.29113899245210073, 0.0)

In [73]:
df_1c.groupby("default-next").count()

,sim#,period,bankID,theta (risk aversion),wealth,deposits,cash,assets,leverage,credit available,...,cash-lag,assets-lag,leverage-lag,credit-available-lag,credit-issued-lag,dummy-0-leverage-lag,over-leverage-frequency,default-next-wealth,default-next-deposit,default-next-interest
default-next,,,,,,,,,,,,,,,,,,,,,
0,18079,18079,18079,18079,18079,18079,18079,18079,18079,18079,...,18079,18079,18079,18079,18079,18079,18079,18079,18079,18079
1,161,161,161,161,161,161,161,161,161,161,...,161,161,161,161,161,161,161,161,161,161


In [129]:
def customized_k_fold(X, y, fold=12):
    """
    customized K-fold cross validation 
    print certain summary statistics
    """
    # k-fold
    kf = KFold(n_splits=fold, shuffle=True)

    # initialization
    accuracy = 0
    conf = np.array([[0, 0], [0, 0]])

    def g(A, arr):
        if A.shape[1] == 1: return A[arr]
        else: return A.loc[arr]
    
    for train_index, test_index in kf.split(X):
        model = LogisticRegression()
        model.fit(g(X,train_index), y.iloc[train_index])
        print(model.coef_, model.intercept_)

        accuracy += model.score(g(X,test_index), y.iloc[test_index])
        conf += confusion_matrix(y.iloc[test_index], model.predict(g(X,test_index)))

    tn, fp, fn, tp = conf.ravel()
    print("{}\n accuracy:{:24}\n precision:{:24}\n recall:{:24}\n".format(
            conf, 
            accuracy/fold, 
            tp/(tp+fp),
            tp/(tp+fn)))

In [130]:
# 1111111111111111111111111
# K-fold cross validation 
# use approx. default probability to predict default
# ###########################
customized_k_fold(overall_prediction.reshape(-1, 1), df_1c["default-next"])

[[0.03405768]] [-6.42528933]
[[0.03473714]] [-6.46272979]
[[0.03380746]] [-6.41389777]
[[0.03355641]] [-6.37083562]
[[0.03450105]] [-6.40688298]
[[0.03404001]] [-6.40668752]
[[0.03413235]] [-6.44832905]
[[0.03435786]] [-6.41148583]
[[0.03490794]] [-6.43499478]
[[0.0343095]] [-6.39732375]
[[0.03367306]] [-6.36780687]
[[0.03459609]] [-6.42041208]
[[18048    31]
 [  157     4]]
 accuracy:      0.9896929824561402
 precision:     0.11428571428571428
 recall:    0.024844720496894408



In [98]:
# prediction based on logistic regression with balanced sheet information
bs_prediction_all = node_weights_all.reshape(1,-1)[0]
bal_sh_prediction = bs_prediction_all [bs_prediction_all != -1]

In [123]:
# zip two predictions into a dataframe
two_predictions = pd.DataFrame({'balanced sheet':bal_sh_prediction, 'network':overall_prediction})

In [131]:
# 222222222222222222222222222
# K-fold cross validation
# use approx. default probability + balanced sheet default prob to predict default
# ###########################
customized_k_fold(two_predictions, df_1c["default-next"])

[[5.04920383 0.00883927]] [-7.48205682]
[[5.2115924  0.00844034]] [-7.54137343]
[[5.32648225 0.00755775]] [-7.55980727]
[[5.27937073 0.00758088]] [-7.56029982]
[[5.29623741 0.00745591]] [-7.52979474]
[[5.42308706 0.00708659]] [-7.61873066]
[[5.42595607 0.0064049 ]] [-7.52656998]
[[5.48055569 0.00695432]] [-7.59585992]
[[5.22089213 0.00860098]] [-7.57119417]
[[5.33261427 0.00746125]] [-7.53848003]
[[5.383764   0.00756245]] [-7.61814698]
[[5.32890542 0.00811476]] [-7.63000265]
[[18079     0]
 [  161     0]]
 accuracy:      0.9911732456140352
 precision:                     nan
 recall:                     0.0



/Users/xcheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in long_scalars


----------------------------------------------------------------------
Stuff Below this are old stuff that I might or might not need.
----------------------------------------------------------------------
----------------------------------------------------------------------

In [125]:
def dist_avg_max(N):
    """
    calculate average & max distances between all pair of nodes
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    
    Returns
    ----------
    avg_d: 2D numpy array [n_simulations, n_periods]
        average distances between all pair of nodes
    max_d: 2D numpy array [n_simulations, n_periods]
        max distances between all pair of nodes
    """
    numSim, numPeriod, _, _ = N.shape
    avg_d = np.empty((numSim, numPeriod-2))
    max_d = np.empty((numSim, numPeriod-2))
    
    for s in range(numSim):
        for p in range(1,numPeriod-1):
            disG = nx.DiGraph(N[s,p])
            dists = shortest_path_length(disG, weight=None)
            curlist=[]
            for source in dists:
                curlist.extend(source[1].values())
            avg_d[s,p-1] = sum(curlist) / float(len(curlist))
            max_d[s,p-1] = max(curlist)
            
    return avg_d, max_d

In [120]:
# ###########################
# Visualize max/avg distances between banks
# ###########################
# avgg, maxx = dist_avg_max(mx_1n)
#
# pavg = pd.DataFrame(avgg)
# pmax = pd.DataFrame(maxx)
# # pavg.mean().plot()
# abc = pmax.stack().value_counts().sort_index().plot(
#     kind="bar",
#     title="max distances, 1 interest rates, 50 simulations, 15 periods",
#     figsize=(8,6),
#     fontsize=12
# )
# abc.set_xlabel("max distance between any pair of reachable nodes")
# abc.set_ylabel("frequncy")
# abc.title.set_fontsize(15)
# abc.xaxis.label.set_fontsize(15)
# abc.yaxis.label.set_fontsize(15)

In [6]:
def weigh_networks(N, model, variables):
    """
    Add weight to network
    Each debt is multiplied by lenders' predicted default probability
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    model: scikit learn LogisticRegression
        model for default probability
    variables: a list of strings
        independent variables for the model
    
    Returns
    ----------
    WN: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        new weighted debt adjacency matrices
    """
    
    WN = np.copy(N)
    simNum, periodNum, bankNum, _= N.shape
    
    for s in range(simNum):
        for p in range(1,periodNum-1):
            for b in range(bankNum):
                X = df_1c[np.array(df_1c["sim#"]==s) &
                          np.array(df_1c["period"]==p) & 
                          np.array(df_1c["bankID"]==b)
                         ][variables].values
                if X.any():
                    predicted_default_probability = model.predict_proba(X)[0][1]
                    WN[s, p-1, b] *= predicted_default_probability
                    
    return WN

In [46]:
def my_pagerank_numpy(G, alpha=0.85, personalization=None, weight='weight', dangling=None):
    """
    This is basically pagerank_numpy without normalization.
    """
    from networkx.algorithms.link_analysis.pagerank_alg import google_matrix
    
    if len(G) == 0:
        return {}
    M = google_matrix(G, alpha, personalization=personalization,
                      weight=weight, dangling=dangling)
    # use numpy LAPACK solver
    eigenvalues, eigenvectors = np.linalg.eig(M.T)
    ind = np.argmax(eigenvalues)
    # eigenvector of largest eigenvalue is at ind
    largest = np.array(eigenvectors[:, ind]).flatten().real
    return dict(zip(G, map(float, abs(largest))))

In [8]:
def apply_to_networks(f, N):
    """
    Calculate Page Rank scores for all the networks 
    
    Parameters
    ----------
    f: function (2D numpy array -> matrix)
        the function to apply to each network (e.g. Page Rank)
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices (netowrks)
    
    Returns
    ----------
    PG: 3D numpy array [n_simulations, n_periods, n_banks]
        Page Rank scores
    """
    
    simNum, periodNum, bankNum, _= N.shape
    PG = np.empty((simNum, periodNum, bankNum))
    
    for s in range(simNum):
        for p in range(1,periodNum-1):
            PG[s, p] = np.array(list(f(nx.DiGraph(N[s, p])).values()))
            
    return PG

In [9]:
# ###########################
# Let's add the weight
# ###########################
weighted = weigh_networks(mx_1n, final, independent)

In [50]:
# ###########################
# Let's calculate pagerank
# ###########################
pg_iter = apply_to_networks(pagerank, weighted)
pg_norm = apply_to_networks(pagerank_numpy, weighted)
pg_not_norm = apply_to_networks(my_pagerank_numpy, weighted)